In [112]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# importing from scikit-learn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

In [113]:
# Load data
df = pd.read_csv('preprocessed_spotify_data_for_prediction.csv')
df.head()

,artist_count,streams,bpm,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,key_sin,key_cos,month_sin,month_cos,day_sin,day_cos,song_age
0,0.25,0.768533,0.447817,1,0.790323,0.921779,0.842857,0.336957,0.0,0.046548,0.024697,0.866025,0.5,-5.000000e-01,-8.660254e-01,0.299363,-0.954139,1
1,0.00,0.764582,0.160811,1,0.645161,0.608895,0.714286,0.076087,0.0,0.077580,0.024697,0.866025,-0.5,1.000000e+00,6.123234e-17,-0.998717,-0.050649,1
2,0.00,0.767838,0.560880,1,0.322581,0.284836,0.414286,0.184783,0.0,0.403413,0.074092,-0.866025,-0.5,1.224647e-16,-1.000000e+00,-0.201299,0.979530,1
3,0.00,0.891451,0.839189,1,0.387097,0.575372,0.685714,0.119565,0.0,0.093095,0.296369,0.000000,1.0,-8.660254e-01,-5.000000e-01,-0.998717,-0.050649,5
4,0.00,0.822617,0.613063,0,0.548387,0.184266,0.800000,0.152174,1.0,0.093095,0.074092,0.000000,1.0,5.000000e-01,-8.660254e-01,-0.485302,-0.874347,1


In [114]:
# Dimensions
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

Number of rows: 952
Number of columns: 18


In [115]:
# Define features and target
y = df['streams']  # Target variable (y)
X = df.drop('streams', axis=1) # Drop target variable and define the features (x)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the MinMaxScaler for consistency
scaler = MinMaxScaler()

# Fit the scaler only on the training data and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Apply the same transformation to the test data (using the same scaler)
X_test_scaled = scaler.transform(X_test)

In [116]:
# --- Baseline Model (Mean Prediction) ---
baseline_pred = np.full(y_test.shape, y_train.mean())
baseline_mae = mean_absolute_error(y_test, baseline_pred)
baseline_mse = mean_squared_error(y_test, baseline_pred)
baseline_r2 = r2_score(y_test, baseline_pred)
print("\nBaseline Model Evaluation:")
print(f"Mean Absolute Error: {baseline_mae:.2f}")
print(f"Mean Squared Error: {baseline_mse:.2f}")
print(f"R² Score: {baseline_r2:.2f}")


Baseline Model Evaluation:
Mean Absolute Error: 0.06
Mean Squared Error: 0.01
R² Score: -0.00


In [117]:
# --- Linear Regression Model ---
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)

# Make predictions
y_pred_lin_reg = lin_reg.predict(X_test_scaled)

# Evaluate Linear Regression model
print("Linear Regression Model Evaluation:")
rmse = np.sqrt(mean_squared_error(y_test, y_pred_lin_reg))
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred_lin_reg):.2f}")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred_lin_reg):.2f}")
print(f"R² Score: {r2_score(y_test, y_pred_lin_reg):.2f}")

Linear Regression Model Evaluation:
Root Mean Squared Error: 0.07
Mean Absolute Error: 0.06
Mean Squared Error: 0.00
R² Score: 0.07


In [118]:
# --- Gradient Boosting Model ---
grad_boost = GradientBoostingRegressor(random_state=42)
grad_boost.fit(X_train_scaled, y_train)

# Make predictions
y_pred_grad_boost = grad_boost.predict(X_test_scaled)

# Evaluate Gradient Boosting model
print("\nGradient Boosting Model Evaluation:")
rmse = np.sqrt(mean_squared_error(y_test, y_pred_grad_boost))
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred_grad_boost):.2f}")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred_grad_boost):.2f}")
print(f"R² Score: {r2_score(y_test, y_pred_grad_boost):.2f}")


Gradient Boosting Model Evaluation:
Root Mean Squared Error: 0.06
Mean Absolute Error: 0.04
Mean Squared Error: 0.00
R² Score: 0.37


In [ ]:
# --- Grid Search for Gradient Boosting ---
param_grid = {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1], 'max_depth': [3, 5], 'subsample': [0.8, 1.0]}
grid_search = GridSearchCV(estimator=grad_boost, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)

print("Best Hyperparameters for Gradient Boosting:", grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test_scaled)

print("\nTuned Gradient Boosting Model Evaluation:")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred_best):.2f}")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred_best):.2f}")
print(f"R² Score: {r2_score(y_test, y_pred_best):.2f}")

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [ ]:
# Calculate regression metrics
metrics = ['Mean Absolute Error', 'Mean Squared Error', 'R² Score']

# Linear Regression metrics
mae_lin_reg = mean_absolute_error(y_test, y_pred_lin_reg)
mse_lin_reg = mean_squared_error(y_test, y_pred_lin_reg)
r2_lin_reg = r2_score(y_test, y_pred_lin_reg)

# Gradient Boosting metrics
mae_grad_boost = mean_absolute_error(y_test, y_pred_grad_boost)
mse_grad_boost = mean_squared_error(y_test, y_pred_grad_boost)
r2_grad_boost = r2_score(y_test, y_pred_grad_boost)

# --- Baseline Model: Mean Prediction (Regression) ---
baseline_pred = np.full_like(y_test, y_train.mean())  # Predict mean of training data
mae_baseline = mean_absolute_error(y_test, baseline_pred)
mse_baseline = mean_squared_error(y_test, baseline_pred)
r2_baseline = r2_score(y_test, baseline_pred)

# --- Gradient Boosting w/ Tuning metrics ---
mae_grad_tuned = mean_absolute_error(y_test, y_pred_best)
mse_grad_tuned = mean_squared_error(y_test, y_pred_best)
r2_grad_tuned = r2_score(y_test, y_pred_best)

# Organize metrics for visualization
baseline_metrics = [mae_baseline, mse_baseline, r2_baseline]
linear_metrics = [mae_lin_reg, mse_lin_reg, r2_lin_reg]
gradient_metrics = [mae_grad_boost, mse_grad_boost, r2_grad_boost]
tuned_metrics = [mae_grad_tuned, mse_grad_tuned, r2_grad_tuned]


# Visualization: Compare Regression Metrics
x = np.arange(len(metrics))  # Metric index
width = 0.2  # Bar width

plt.figure(figsize=(10, 6))
plt.bar(x - (2 * width), baseline_metrics, width, label='Baseline (Mean)', color='gray')
plt.bar(x - width, linear_metrics, width, label='Linear Regression', color='blue')
plt.bar(x, gradient_metrics, width, label='Gradient Boosting', color='yellow')
plt.bar(x + width, tuned_metrics, width, label='GridSearchCV', color='green')
plt.bar

# Add metric names and labels
plt.title('Model Comparison: Regression Metrics')
plt.ylabel('Score')
plt.xticks(x, metrics)
plt.legend()
plt.tight_layout()

# Annotate scores above bars
for i in range(len(metrics)):
    plt.text(x[i] - (width * 2), baseline_metrics[i] + 0.01, f"{baseline_metrics[i]:.2f}", ha='center')
    plt.text(x[i] - width, linear_metrics[i] + 0.01, f"{linear_metrics[i]:.2f}", ha='center')
    plt.text(x[i], gradient_metrics[i] + 0.01, f"{gradient_metrics[i]:.2f}", ha='center')
    plt.text(x[i] + width, tuned_metrics[i] + 0.01, f"{tuned_metrics[i]:.2f}", ha='center')

plt.show()


In [ ]:
# --- Feature Importance ---
plt.figure(figsize=(10, 6))

# Get Feature Importances
feature_importances = pd.Series(grad_boost.feature_importances_, index=X.columns)
feature_importances_sorted = feature_importances.sort_values(ascending=False)
ax = feature_importances_sorted.plot(kind='bar', color='skyblue')

# Annotate the importance values at the top of each bar
for index, value in enumerate(feature_importances_sorted):
    plt.text(index, value + 0.002, f"{value:.2f}", ha='center', va='bottom', fontsize=8)

plt.title('Feature Importances (Gradient Boosting)', fontsize=14)
plt.ylabel('Importance', fontsize=12)
plt.xlabel('Features', fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
# Residual Plot for Gradient Boosting
residuals = y_test - y_pred_grad_boost
plt.figure(figsize=(8, 6))
plt.scatter(y_pred_grad_boost, residuals, color='green', alpha=0.6)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residual Plot for Gradient Boosting')
plt.xlabel('Predicted Streams')
plt.ylabel('Residuals')
plt.show()

In [ ]:
# Check some examples of actual vs predicted values
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_grad_boost})
print(comparison_df.head(10))


In [ ]:
# Plotting actual vs predicted values for both models
plt.figure(figsize=(12, 6))

# Linear Regression
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred_lin_reg, color='blue', label='Predictions')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Ideal Line')
plt.title('Linear Regression: Actual vs Predicted')
plt.xlabel('Actual Popularity')
plt.ylabel('Predicted Popularity')
plt.legend()

# Gradient Boosting
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred_grad_boost, color='orange', label='Predictions')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Ideal Line')
plt.title('Gradient Boosting: Actual vs Predicted')
plt.xlabel('Actual Popularity')
plt.ylabel('Predicted Popularity')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Plotting actual vs predicted values for both models
plt.figure(figsize=(12, 6))

# GridSearchCV Tuned
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred_best, color='green', label='Predictions')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Ideal Line')
plt.title('GridSearchCV-Tuned: Actual vs Predicted')
plt.xlabel('Actual Popularity')
plt.ylabel('Predicted Popularity')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Calculate the correlation matrix
correlation_matrix = df.corr()

# Visualize the correlation between features and the target variable ('streams')
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix[['streams']].sort_values(by='streams', ascending=False), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation between Features and Streams')
plt.show()

In [ ]:
# Compute correlation heatmap
corr_matrix = df.corr()

# Visualize the correlation heatmap
plt.figure(figsize=(20, 15))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm')
plt.title("Feature Correlation Matrix")
plt.show()

# Prints Top 10 strongest correlations
corr_pairs = corr_matrix.unstack()
sorted_corr = corr_pairs.sort_values(key=abs, ascending=False)
sorted_corr = sorted_corr[sorted_corr != 1]  # Exclude the correlation of a feature with itself
print(sorted_corr.head(10))

In [ ]:
# Plot histograms of streams to check for skewness
plt.figure(figsize=(10, 6))
plt.hist(y, bins=30, edgecolor='black')  # Adjust the number of bins as needed
plt.title('Histogram of Streams')
plt.xlabel('Streams')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Histogram of Gradient Boosting Predictions
plt.figure(figsize=(10, 6))
plt.hist(y_pred_grad_boost, bins=30, edgecolor='black')
plt.title('Distribution of Predicted Streams (Gradient Boosting)')
plt.xlabel('Predicted Streams')
plt.ylabel('Frequency')
plt.show()
